In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Read data
# Change the path to data on your computer
data = pd.read_csv('Analysis-Portfolio-Task-Data.csv')
# Print the data describe
data.describe()

,CustomerID,Shopping Basket,Age,Value Products,Brand Products,Top Fresco Products
count,7.500000e+01,75.000000,75.000000,75.000000,75.00000,75.000000
mean,5.528687e+07,60.960667,38.640000,15.813333,8.60000,6.893333
std,1.990541e+07,39.770647,13.808184,12.213963,6.17974,5.326054
min,2.035806e+07,3.400000,19.000000,0.000000,0.00000,0.000000
25%,3.654195e+07,25.595000,25.500000,7.000000,3.00000,3.000000
50%,5.230947e+07,58.060000,38.000000,13.000000,8.00000,6.000000
75%,7.258872e+07,85.160000,48.500000,20.000000,13.00000,10.000000
max,9.966645e+07,159.470000,68.000000,48.000000,23.00000,20.000000


In [3]:
# Print the data
pd.DataFrame(data)

,CustomerID,Shopping Basket,Gender,Age,Store Type,Value Products,Brand Products,Top Fresco Products
0,20358063,48.81,Male,26,Convenient Stores,8,2,1
1,24635139,33.44,Female,33,Superstore,6,5,1
2,27584479,131.57,Male,56,Online,35,8,12
3,28008212,20.02,Male,27,Convenient Stores,0,1,1
4,29130973,95.54,Female,55,Online,38,18,20
...,...,...,...,...,...,...,...,...
70,86596295,75.23,Female,53,Superstore,9,14,5
71,88907963,97.21,Female,43,Superstore,18,9,10
72,89074559,77.65,Female,46,Superstore,10,10,9
73,95961294,128.67,Female,48,Superstore,41,15,19


In [4]:
# Remove unecessary column
data.drop('CustomerID', axis=1, inplace=True)
# One-hot coding 
store_type_dummies = pd.get_dummies(data['Store Type'], prefix='Store Type')
data.drop('Store Type', axis=1, inplace=True)
data = pd.concat([data, store_type_dummies], axis=1)
data['Gender'] = data['Gender'].map({'Male': 1, 'Female': 0})
pd.DataFrame(data)
# Standardize data
# data['Gender'] = StandardScaler().fit_transform(data['Gender'])

,Shopping Basket,Gender,Age,Value Products,Brand Products,Top Fresco Products,Store Type_Convenient Stores,Store Type_Online,Store Type_Superstore
0,48.81,1,26,8,2,1,True,False,False
1,33.44,0,33,6,5,1,False,False,True
2,131.57,1,56,35,8,12,False,True,False
3,20.02,1,27,0,1,1,True,False,False
4,95.54,0,55,38,18,20,False,True,False
...,...,...,...,...,...,...,...,...,...
70,75.23,0,53,9,14,5,False,False,True
71,97.21,0,43,18,9,10,False,False,True
72,77.65,0,46,10,10,9,False,False,True
73,128.67,0,48,41,15,19,False,False,True


In [5]:
x_data = data.drop('Shopping Basket', axis=1)
y_data = data['Shopping Basket']
# Split training set and validation set
x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                    y_data,
                                                    test_size=0.2,
                                                    random_state=42)
y_train_perceptron = np.asarray(y_train.apply(lambda x: 1 if x >= 50 else -1))
y_test_perceptron = np.asarray(y_test.apply(lambda x: 1 if x >= 50 else -1))

y_train_binary = np.asarray(y_train.apply(lambda x: 1 if x >= 50 else 0))
y_test_binary = np.asarray(y_test.apply(lambda x: 1 if x >= 50 else 0))

In [6]:
# Perceptron approach
# Train model
perceptron = Perceptron()
perceptron.fit(x_train, y_train_perceptron)
# Predict results
y_pred_perceptron = perceptron.predict(x_test)
# Print accuracy score, recall score, precision score
print(f'Accuracy score: {accuracy_score(y_test_perceptron, y_pred_perceptron)}')
print(f'Recall score: {recall_score(y_test_perceptron, y_pred_perceptron)}')
print(f'Precision score: {precision_score(y_test_perceptron, y_pred_perceptron)}')
print(f'Coefficient: {perceptron.coef_}')

Accuracy score: 0.6
Recall score: 0.3333333333333333
Precision score: 1.0
Coefficient: [[-41. -75.  48. 144.  48. -54.  21. -16.]]


**Nhận xét:**
- Accuracy (0.6) cho thấy mô hình dự đoán đúng khoảng 60% các trường hợp. Recall (0.333) rất thấp, có nghĩa là mô hình bỏ sót nhiều trường hợp thực sự thuộc về lớp dương (false negatives cao). Tuy nhiên, Precision (1.0) cho thấy rằng các dự đoán dương đều chính xác (không có false positives).
- Hệ số cho thấy sự tác động mạnh của các biến đến kết quả (ví dụ như các hệ số lớn như 144 hay -75). Perceptron có thể chưa tối ưu trong việc học từ dữ liệu này vì hệ số lớn và kết quả không cân bằng giữa recall và precision.

In [7]:
# Logistic Regression approach
# Train model
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train, y_train_binary)
# Predict results
y_pred_logistic = logistic_regression.predict(x_test)
# Print accuracy score, recall score, precision score
print(f'Accuracy score: {accuracy_score(y_test_binary, y_pred_logistic)}')
print(f'Recall score: {recall_score(y_test_binary, y_pred_logistic)}')
print(f'Precision score: {precision_score(y_test_binary, y_pred_logistic)}')
print(f'Coefficient: {logistic_regression.coef_}')

Accuracy score: 1.0
Recall score: 1.0
Precision score: 1.0
Coefficient: [[ 0.10151414  0.08804892  0.27124597  0.57077996 -0.04163791 -0.69280689
   0.34674773  0.34569683]]


**Nhận xét:**
- Cả Accuracy, Recall và Precision đều hoàn hảo (1.0), cho thấy rằng mô hình Logistic Regression đã học tốt từ dữ liệu và phân loại chính xác tất cả các mẫu.
- Hệ số trong Logistic Regression tương đối nhỏ, điều này phản ánh rằng các biến tác động nhẹ đến kết quả. Tuy nhiên, biến số có hệ số cao nhất là 0.57077996 cho thấy sự tác động mạnh mẽ nhất trong việc phân loại.

In [8]:
# Naive Bayes approach
# Train model
naive_bayes = GaussianNB()
naive_bayes.fit(x_train, y_train_binary)
# Predict results
y_pred_naive = naive_bayes.predict(x_test)
# Print accuracy score, recall score, precision score
print(f'Accuracy score: {accuracy_score(y_test_binary, y_pred_naive)}')
print(f'Recall score: {recall_score(y_test_binary, y_pred_naive)}')
print(f'Precision score: {precision_score(y_test_binary, y_pred_naive)}')

Accuracy score: 0.8666666666666667
Recall score: 0.7777777777777778
Precision score: 1.0


**Nhận xét:**
- Accuracy (0.867) cho thấy Naive Bayes dự đoán đúng phần lớn các mẫu. Recall (0.778) cao hơn so với Perceptron, có nghĩa là mô hình nhận diện tốt hơn các mẫu thực sự dương, nhưng không cao bằng Logistic Regression. Precision (1.0) vẫn rất tốt, cho thấy các dự đoán dương đều đúng.
- Naive Bayes hoạt động tốt nhưng không hoàn hảo như Logistic Regression, có thể vì giả định về tính độc lập giữa các biến không được thỏa mãn hoàn toàn trong dữ liệu.